# 1.) Load data into dataframe and produce mean-varied distributions ("modified LMPs")

In [ ]:
import pandas as pd
dynamic = pd.read_csv('Martinez_2020.csv') # use appropriate .csv
dynamic

,Unnamed: 0,$/kWh,Unnamed: 2,Unnamed: 3,OPR_DT,OPR_HR
0,1,0.033740,NaN,NaN,1/1/2020,1
1,2,0.031798,NaN,NaN,1/1/2020,2
2,3,0.033113,NaN,NaN,1/1/2020,3
3,4,0.032264,NaN,NaN,1/1/2020,4
4,5,0.032171,NaN,NaN,1/1/2020,5
...,...,...,...,...,...,...
8778,8779,0.050840,NaN,NaN,12/31/2020,20
8779,8780,0.049190,NaN,NaN,12/31/2020,21
8780,8781,0.044183,NaN,NaN,12/31/2020,22
8781,8782,0.040715,NaN,NaN,12/31/2020,23


In [ ]:
dfElPrice = dynamic.iloc[:,1]
dfElPrice
Life_1year = len(dfElPrice)
Life_1year

8783

In [ ]:
dfElPrice.describe()

count    8783.000000
mean        0.021946
std         0.036115
min         0.003310
25%         0.013905
50%         0.017510
75%         0.023390
max         2.015860
Name: $/kWh, dtype: float64

In [ ]:
# allows user to input a desired mean
# transforms all values in the data frame to correspond to this new mean

def varyMean(desired_mean, df):
    original_mean = df.mean()
    delta = desired_mean - original_mean
    data_list = []
    for i in range(len(df)):
        data = df[i]
        new_data = data + delta
        data_list.append(new_data)
    new_df = pd.DataFrame(data_list)[0]
    return new_df

In [ ]:
# desired means: 0.00 to 0.07 $/kWh
# create numpy arrays with mean-transformed datasets 

import numpy as np
mean_array = np.linspace(0,0.07,num=8)
mean_array
df_array = []
for mean in mean_array:
    df_array.append(varyMean(mean, dfElPrice))

df_array_dict = {}
for i in range(len(mean_array)):
    df_array_dict[str(mean_array[i])]=df_array[i]
df_array_dict.keys()

# test the length of a sample array and display values
Life_1year = len(df_array_dict['0.0'])
df_array_dict['0.04']

0       0.039866
1       0.037923
2       0.039239
3       0.038390
4       0.038297
          ...   
8778    0.056966
8779    0.055316
8780    0.050309
8781    0.046841
8782    0.045484
Name: 0, Length: 8783, dtype: float64

# 2.) Create 10 years worth of electricity prices for each distribution

In [ ]:
# use dfElPrice (original dataset, not modified LMPs) for sensitivity analysis
# create 10 years of locational marginal prices (LMPs) by appending the dataset 
# to itself

df2 = dfElPrice
for i in range(9):
    test = dfElPrice.append(df2)
    dfElPrice = test
test
dynamic = test 
dynamic
Life_hours = len(dynamic) 

In [ ]:
# use modified LMPs to run mode comparisons
# create 10 years of locational marginal prices (LMPs) by appending the dataset 
# to itself

for mean in df_array_dict:
    df2 = df_array_dict[mean]
    for i in range(9):
        test = df_array_dict[mean].append(df2)
        df_array_dict[mean] = test
    test
    dynamic = test 
    dynamic
    Life_hours = len(dynamic)

# 3.) Mode 1A - Constant 1.7 Jop

In [ ]:
import math  # for the exponential function
import numpy as np
import matplotlib.pyplot as plt

# write results to csv
f_file = open("figure1a.csv","w")
print("Mean Price","Jop","LCOH","UNINST CAPperkW","INST CAPEX", "OPEX","OM","PV_Costs", "PV_KG", file=f_file,sep=",")

# runs Mode 1A for all datasets 
for mean in df_array_dict: 
    print("Mean Price:", mean)

    # Technical Variables 
    Jop_rated = 1.7 # A/cm2
    Vop_rated = 1.743136145279 # Volts
    Capacity = 1000  # conversion factor from kW to watts
    Capacity_kW = 10000 #kW
    Stack_cost_1kW = 473.92 #2020 USD 
    M_BOP_1kW = 150.10 #2020 USD 
    E_BOP_1kW = 130.48 #2020 USD 
    CAPEX_per_1kW = Stack_cost_1kW + M_BOP_1kW + E_BOP_1kW #uninstalled cost = 754.5
    
    # changing capex with current density
    Electronics_percentage = 0.26 # cost increase
    # for electrical balance of plant and phase separator and dryers
    CAPEX_Electronics = Electronics_percentage*CAPEX_per_1kW
    CAPEX_Rest = (1-Electronics_percentage)*CAPEX_per_1kW
    A = (Capacity * Capacity_kW) / (Vop_rated * Jop_rated) #cm2 electrode area
    CAPEX_Rest_Total = CAPEX_Rest*A*Jop_rated*Vop_rated/1000 #1000 converts to kW

    Jop_ = 1.7
    Y = lambda X: ((0.156726387554 * X) + 1.47670128643691) #x is Jop_
    while Jop_ <= 1.7: # 1.7 A/cm2 is only available Jop for Mode 1A
        print("Jop_:", Jop_, "Jop_rated:", Jop_rated, "Vop_rated:", Vop_rated)
        Vop_ = Y(Jop_) 

        n_mol = 2  # constant for H2
        F = 96485  # Faraday's constant
        V_threshold = 2.9  # voltage at which current removed from options

        N_life = 10  # life of electrolyzer
      
        DR = 0.10  # discount rate

        # vary A/cm2 in 0.1 increments 0 to 6.0 
        Jop = [Jop_] 
        print("Jop:",Jop[0])
        
        Vop = [Vop_] #1.79
        print("Vop:",Vop[0])
    
        PReq = []  # kW required per hour
        for i in range(len(Jop)):  
            if Jop[i] != 0:
                PReq.append((A * Jop[i] * Vop[i]) / 1000)  # kW
            else:
                PReq.append(0)

        # degrade voltage
        Degradation_Rate = 0.0015 # 0.0015 

        # prevents Jop[i] and Vop[i] from being modified
        def updatePReq(i, Jop_=Jop[i], Vop_=Vop[i]):  

            if Jop[i] != 0:
                PReq[i] = ((A * Jop_ * Vop_) / 1000)  # kW
            else:
                PReq[i] = 0
            return PReq[i]

        Vop_copy = Vop[:]  # stores degraded voltage
        Energy_Charge = {}  
        Degraded_Voltage = {} # testing
        threshold_list_J = []  # at which current is the threshold reached
        threshold_list_hour = []  # at which hour is threshold reached
        Jop_plotting = []  
        Jop_fallback = []
        Vop_plotting = [] 
        Vop_fallback = []  
        for i in range(len(Jop)):  
            temp_EC = []
            temp_degraded_voltage = [] # testing to accumulate degraded voltage
            count = 0
            current_index = i
            Jop_temp = Jop[i]
            Vop_temp = Vop[i]
            hour = 0  # first hour is 1
            Degrade_count = 0  # 10.14.20 - counts how many times degradation 
            #has been done to apply to the new voltage
            for price in df_array_dict[mean]:  
                count += 1
                hour += 1
                if count == 1000:
                    Degrade_count += 1
                    count = 0
                    if Jop[i] != 0:
                        # for each current, cycle through hours, 
                        # check after each 1k hours the new voltage
                        # Check if voltage above threshold. 
                        # If so, choose the current which is directly 
                        # below it as range maximum
                        #if hour is less than 1000 use IV curve, 
                        # if over 1k hr use new values
                        current_index = i  # voltage currently operating at
                        # 10.14.20 - applies DR every 1k hrs
                        Vop_copy[i] += Vop_copy[i] * Degradation_Rate  
                        if Vop_copy[i] >= V_threshold:
                            Vop_temp = Vop_copy[i]  # use the degraded voltage
                            temp_EC.append(np.nan)  
                            temp_degraded_voltage.append(Vop_copy[i])                 
                        else:  # threshold not reached
                            Vop_temp = Vop_copy[i]  # use the degraded voltage
                            # calculate Energy Charge with degraded voltage
                            temp_EC.append(
                                price * updatePReq(i, Jop[i], Vop_copy[i]))  
                            temp_degraded_voltage.append(Vop_copy[i]) #for printing         
                        if Jop[
                            current_index] not in threshold_list_J:  
                            threshold_list_J.append(Jop[current_index])
                            threshold_list_hour.append(hour)
                        Vop_plotting.append(Vop[i])
                        Vop_fallback.append(Vop_temp)
                        Jop_plotting.append(Jop[i])
                        Jop_fallback.append(Jop[current_index])
                    else:
                        temp_EC.append(
                            0)  
                        temp_degraded_voltage.append(Vop_copy[i]) 
                    # collects Energy Charges in a dictionary         
                    Energy_Charge["EC_Jop" + str(round(Jop[i], 2))] = temp_EC 
                    Degraded_Voltage["DV_Jop" + str(round(Jop[i], 2))] = temp_degraded_voltag
                else:  # anything other than thousands (e.g. 1 to 999 and 1001 to 1999, 2001 t0 2999)
                    if Jop[i] != 0:
                        if Vop_copy[i] >= V_threshold:
                            temp_EC.append(np.nan)
                            temp_degraded_voltage.append(Vop_copy[i])
                        else:
                            temp_EC.append(price * updatePReq(i, Jop[i], Vop_copy[i]))
                            temp_degraded_voltage.append(Vop_copy[i])
                    else:
                        temp_EC.append(0)  # EC IS $0 for Jop0
                        temp_degraded_voltage.append(Vop_copy[i])
                    Energy_Charge["EC_Jop" + str(round(Jop[i], 2))] = temp_EC
                    Degraded_Voltage["DV_Jop" + str(round(Jop[i], 2))] = temp_degraded_voltage
                
        df_LCOH = pd.DataFrame(data=Energy_Charge)  # holds all energy charges over 10 years
        df_LCOH_total_list = []  # holds all of 10 individual dataframess, 1 for each year
        start = 0
        stop = Life_1year
        for i in range(10):
            df_LCOH_temp = df_LCOH.iloc[start:stop]
            df_LCOH_total_list.append(df_LCOH_temp)  # saves each year in a list so can loop through
            start = stop
            stop = (i + 2) * Life_1year # cycles though each year

        for df in df_LCOH_total_list:
            df.index = range(Life_1year)
 
        M_CAPEX = []  # varying the CAPEX_Electronics with Jop
        for i in range(len(Jop)):
            if Jop[i] > 1.7:
                CAPEX_Electronics_Total =  CAPEX_Electronics*A*Jop[i]*Vop[i]/1000
                Installation_Factor = 1.19 #Installed CAPEX 
                Total_CAPEX = (CAPEX_Rest_Total + CAPEX_Electronics_Total)*Installation_Factor
                M_CAPEX.append(Total_CAPEX/Life_hours) 
            elif Jop[i] > 0 and Jop[i] <= 1.7:
                CAPEX_Electronics_Total =  CAPEX_Electronics*A*Jop_rated*Vop_rated/1000
                Installation_Factor = 1.19
                Total_CAPEX = (CAPEX_Rest_Total + CAPEX_Electronics_Total)*Installation_Factor
                M_CAPEX.append(Total_CAPEX/Life_hours) 
            else:
                M_CAPEX.append(0)  
          
        M_OM = [x*0.05*10 for x in M_CAPEX] # Use * 10 to correct 
        # because M_OM is taking 5% of M_CAPEX, which is based on 
        # the total CAPEX / life_hours. For Year 
        # 1 OM need the sum of 1 year hours M_OM and NOT lifetime hours 

        # Rate of fuel production in kg every hour of year 1
        M_kgh2 = []
        for i in range(len(Jop)):
            if Jop[i] != 0:
                M_kgh2.append((Jop[i] * A) / (n_mol * F) * (0.002 * 3600))  
            else:
                M_kgh2.append(0)  # 0 kg produced when Jop=0

        for df_LCOH in df_LCOH_total_list: # cycle through and get each year
            for i in range(len(Jop)):
                df_LCOH["kg_Jop" + str(round(Jop[i], 2))] = [np.nan if np.isnan(df_LCOH["EC_Jop" + str(round(Jop[i], 2))][j]) \
                                                                else M_kgh2[i] for j in range(
                    Life_1year)]  # list, if EC for that hour and current is NaN, make Nan, else use the calc
                df_LCOH["CAP_Jop" + str(round(Jop[i], 2))] = [np.nan if np.isnan(df_LCOH["EC_Jop" + str(round(Jop[i], 2))][j]) \
                                                                  else M_CAPEX[i] for j in range(Life_1year)]
                df_LCOH["OM_Jop" + str(round(Jop[i], 2))] = [np.nan if np.isnan(df_LCOH["EC_Jop" + str(round(Jop[i], 2))][j]) \
                                                                else M_OM[i] for j in range(Life_1year)]

        col1 = []
        for df_LCOH in df_LCOH_total_list:
            for i in range(len(Jop)):  
                CAP_name = 'CAP_Jop' + str(round(Jop[i], 2))
                OM_name = 'OM_Jop' + str(round(Jop[i], 2))
                EC_name = 'EC_Jop' + str(round(Jop[i], 2))
                kg_name = 'kg_Jop' + str(round(Jop[i], 2))

                col1.append(CAP_name)
                col1.append(OM_name)
                col1.append(EC_name)
                col1.append(kg_name)

            df_LCOH = df_LCOH[col1]

        # Compare all columns and pick up the minimum LCOH - no comparison for mode 1A
        for df_LCOH in df_LCOH_total_list:  
            for i in range(len(Jop)):
                    df_LCOH["mLCOH_Jop" + str(round(Jop[i], 2))] = (df_LCOH["CAP_Jop" + str(round(Jop[i], 2))] + df_LCOH[
                        "OM_Jop" + str(round(Jop[i], 2))] + df_LCOH["EC_Jop" + str(round(Jop[i], 2))]) / df_LCOH[
                                                                      "kg_Jop" + str(round(Jop[i], 2))]

            mLCOH_title_list = []

            for i in range(len(Jop)):  
                mLCOH_title_list.append("mLCOH_Jop" + str(round(Jop[i], 2)))

            df_LCOH["mLCOH_min"] = df_LCOH[mLCOH_title_list].min(axis=1)
          
            df_LCOH[mLCOH_title_list].loc[
                0].min()
            df_LCOH


        # Calculate annual lcoh by summing the energy charge associated with the
        # hourly mLCOH 

        OptimalEC_List_DCFA = []  # for all 10 years
        for df_LCOH in df_LCOH_total_list:
            OptimalEC_Year1 = 0  
            OptimalJop_Year1_List = []
            OptimalEC_Year1_List = []

            # we take length of the year; i is along row for all hrs in yr
            for i in range(len(df_LCOH["mLCOH_Jop"+str(round(Jop_,2))])):  
                mLCOH_list = []
                for k in range(len(Jop)):  # k collects Jop values into list 
                    mLCOH_list.append(df_LCOH["mLCOH_Jop" + str(round(Jop[k], 2))][i])
                temp_mLCOH_min = min(mLCOH_list)

                # then for the minimum LCOH find the location inside the list
                for j in range(len(mLCOH_list)):
                    if temp_mLCOH_min == mLCOH_list[j]:  # take the index from the min LCOH
                        index = j  # save the position for which it is associated so it can be used in next line
                if i == 0: 
                    pass
                # sum the corresponding energy charge for the mLCOH_min
                OptimalEC_Year1 += df_LCOH["EC_Jop" + str(round(Jop[index], 2))][i]  # sums the corresponding min EC
                OptimalEC_Year1_List.append(df_LCOH["EC_Jop" + str(round(Jop[index], 2))][i])  # adds all to df
                OptimalJop_Year1_List.append(Jop[index])

            OptimalEC_List_DCFA.append(OptimalEC_Year1)  

            df_LCOH["m_ECmin"] = OptimalEC_Year1_List  # creates the df column
            df_LCOH["OptimalJop"] = OptimalJop_Year1_List

        # for the mLCOH find the corresponding kg, put in a new column and sum
        OptimalKG_List_DCFA = []  
        for df_LCOH in df_LCOH_total_list:
            OptimalKG_Year1 = 0
            # cycle through each row and find the minimum LCOH
            OptimalKG_Year1_List = []
            for i in range(len(df_LCOH["mLCOH_Jop"+str(round(Jop_,2))])): 
                mLCOH_list = []
                for k in range(len(Jop)):  
                    mLCOH_list.append(df_LCOH["mLCOH_Jop" + str(round(Jop[k], 2))][i])
                temp_mLCOH_min = min(mLCOH_list)

                # then for the minimum LCOH find the location inside the list
                for j in range(len(mLCOH_list)):
                    if temp_mLCOH_min == mLCOH_list[j]:  # take the index from the min LCOH
                        index = j  # save the position for which it is associated so it can be used next line
                if i == 0:  
                    pass
                OptimalKG_Year1 += df_LCOH["kg_Jop" + str(round(Jop[index], 2))][i]  # sums the corresponding min kg
                OptimalKG_Year1_List.append(df_LCOH["kg_Jop" + str(round(Jop[index], 2))][i])  # adds all to list to check

            OptimalKG_List_DCFA.append(OptimalKG_Year1) 

            df_LCOH["m_kgmin"] = OptimalKG_Year1_List

        for df_LCOH in df_LCOH_total_list: 

            # CAPEX YEAR 1
            CAPEX_Year1 = df_LCOH["CAP_Jop"+str(round(Jop_,2))].sum() 

            # OM YEAR 1
            OM_Year1 = df_LCOH["OM_Jop"+str(round(Jop_,2))].sum()

            # LCOH MIN for year 1
            LCOHmin_Year1 = (CAPEX_Year1 + OM_Year1 + OptimalEC_Year1) / OptimalKG_Year1
      
        df_DCFA = pd.DataFrame(data={})
        df_DCFA["Year"] = list(range(0, 11))
        
        df_DCFA["CAPEX"] = [Total_CAPEX if i == 0 else 0 for i in range(11)]
  
        df_DCFA["PWF"] = [1 / (1 + DR) ** df_DCFA["Year"][i] for i in range(11)]

        df_DCFA["PV_OM"] = [0 if i == 0 else (OM_Year1 * df_DCFA["PWF"][i]) for i in range(11)]

        OptimalEC_List_DCFA.insert(0, 0)
        df_DCFA["PV_EC"] = [0 if i == 0 else (OptimalEC_List_DCFA[i] * df_DCFA["PWF"][i]) for i in range(11)]

        OptimalKG_List_DCFA.insert(0, 0)
        df_DCFA["PV_KG"] = [0 if i == 0 else (OptimalKG_List_DCFA[i] * df_DCFA["PWF"][i]) for i in range(11)]

        # Calculate LCOH

        PV_Costs = (df_DCFA["CAPEX"].sum() + df_DCFA["PV_OM"].sum() + df_DCFA[
            "PV_EC"].sum())  

        PV_KG = df_DCFA["PV_KG"].sum()
        Lifetime_LCOH_dynamic_Jop0baseline = PV_Costs / PV_KG

        # Contribution analysis

        CAPEX_Cont = df_DCFA["CAPEX"].sum() / PV_Costs * 100

        OM_Cont = df_DCFA["PV_OM"].sum() / PV_Costs * 100

        EC_Cont = df_DCFA["PV_EC"].sum() / PV_Costs * 100

        df_DCFA["PV_KG"].sum()

        # write to csv
        print(mean, Jop_, Lifetime_LCOH_dynamic_Jop0baseline, CAPEX_per_1kW, Total_CAPEX, df_DCFA["PV_EC"].sum(), df_DCFA["PV_OM"].sum(), PV_Costs, PV_KG, file=f_file,sep=",") 
        Jop_ += 0.1 #the step interval for Jop_rated; must be down here 
        print("UNINST CAPperKW:", CAPEX_per_1kW)
        print("INST CAPEX:", Total_CAPEX)
        print("PV_EC:", df_DCFA["PV_EC"].sum())
        print("PV_KG:", PV_KG)
        print("PV_OM:", df_DCFA["PV_OM"].sum())
        print("LCOH:", Lifetime_LCOH_dynamic_Jop0baseline)
f_file.close() 
df_csv = pd.read_csv("figure1a.csv")
df_csv

Mean Price: 0.0
Jop_: 1.7 Jop_rated: 1.7 Vop_rated: 1.743136145279
Jop: 1.7
Vop: 1.74313614527871
UNINST CAPperKW: 754.5
INST CAPEX: 8978550.0
PV_EC: 13617.665441489133
PV_KG: 11551691.058712926
PV_OM: 2758465.149346238
LCOH: 1.017221872976316
Mean Price: 0.01
Jop_: 1.7 Jop_rated: 1.7 Vop_rated: 1.743136145279
Jop: 1.7
Vop: 1.74313614527871
UNINST CAPperKW: 754.5
INST CAPEX: 8978550.0
PV_EC: 5718800.828730562
PV_KG: 11551691.058712926
PV_OM: 2758465.149346238
LCOH: 1.5111048148150272
Mean Price: 0.02
Jop_: 1.7 Jop_rated: 1.7 Vop_rated: 1.743136145279
Jop: 1.7
Vop: 1.74313614527871
UNINST CAPperKW: 754.5
INST CAPEX: 8978550.0
PV_EC: 11423983.992019637
PV_KG: 11551691.058712926
PV_OM: 2758465.149346238
LCOH: 2.004987756653738
Mean Price: 0.03
Jop_: 1.7 Jop_rated: 1.7 Vop_rated: 1.743136145279
Jop: 1.7
Vop: 1.74313614527871
UNINST CAPperKW: 754.5
INST CAPEX: 8978550.0
PV_EC: 17129167.15530871
PV_KG: 11551691.058712926
PV_OM: 2758465.149346238
LCOH: 2.4988706984924494
Mean Price: 0.04
Jop_

,Mean Price,Jop,LCOH,UNINST CAPperkW,INST CAPEX,OPEX,OM,PV_Costs,PV_KG
0,0.00,1.7,1.017222,754.5,8978550.0,1.361767e+04,2.758465e+06,1.175063e+07,1.155169e+07
1,0.01,1.7,1.511105,754.5,8978550.0,5.718801e+06,2.758465e+06,1.745582e+07,1.155169e+07
2,0.02,1.7,2.004988,754.5,8978550.0,1.142398e+07,2.758465e+06,2.316100e+07,1.155169e+07
3,0.03,1.7,2.498871,754.5,8978550.0,1.712917e+07,2.758465e+06,2.886618e+07,1.155169e+07
4,0.04,1.7,2.992754,754.5,8978550.0,2.283435e+07,2.758465e+06,3.457137e+07,1.155169e+07
5,0.05,1.7,3.486637,754.5,8978550.0,2.853953e+07,2.758465e+06,4.027655e+07,1.155169e+07
6,0.06,1.7,3.980520,754.5,8978550.0,3.424472e+07,2.758465e+06,4.598173e+07,1.155169e+07
7,0.07,1.7,4.474402,754.5,8978550.0,3.994990e+07,2.758465e+06,5.168691e+07,1.155169e+07


In [ ]:
# mode 1A - use original dataset for sensitivity analysis
import math  # for the exponential function
import numpy as np
import matplotlib.pyplot as plt

#1.9.21 - write results to csv
f_file = open("figure1a.csv","w")
print("Jop","LCOH", "UNINST_CAPEXperKW", "INST_CAPEX", "OPEX","OM","PV_Costs", "PV_KG", file=f_file,sep=",")

# Technical Variables
Jop_rated = 1.7  
Vop_rated = 1.743136145279 
Capacity = 1000  # conversion factor
Capacity_kW = 10000 # kW
Stack_cost_1kW = 473.92 #2020 USD 
M_BOP_1kW = 150.10 #2020 USD 
E_BOP_1kW = 130.48 #2020 USD 
CAPEX_per_1kW = Stack_cost_1kW + M_BOP_1kW + E_BOP_1kW  #uninstalled cost = 754.5
Electronics_percentage = 0.26 
CAPEX_Electronics = Electronics_percentage*CAPEX_per_1kW
CAPEX_Rest = (1-Electronics_percentage)*CAPEX_per_1kW
A = (Capacity * Capacity_kW) / (Vop_rated * Jop_rated)  
CAPEX_Rest_Total = CAPEX_Rest*A*Jop_rated*Vop_rated/1000 # 1000 converts to kW

# use efficiency desired for sensitivity analysis
Jop_ = 1.7
# Y = lambda X: ((0.197455703 * X) + 1.522282695) #x is Jop_ - low efficiency
# Y = lambda X: ((0.11024708 * X) + 1.503083808) #x is Jop_ - high efficiency
Y = lambda X: ((0.156726387554 * X) + 1.47670128643691) #x is Jop_ baseline efficiency
while Jop_ <= 1.7: #keep increase at one decimal point increase
    print("Jop_:", Jop_, "Jop_rated:", Jop_rated, "Vop_rated:", Vop_rated)
    Vop_ = Y(Jop_) 

    n_mol = 2  # constant for H2
    F = 96485  # Faraday's constant
    V_threshold = 2.9  # 10.9.20 - voltage at which current removed from options
   
    # Financial Variables
    N_life = 10  # life of electrolyzer
    DR = 0.10  # discount rate

    Jop = [Jop_] 
    
    Vop = [Vop_] #1.79
   
    PReq = []  # kW required per hour
    for i in range(len(Jop)):  
        if Jop[i] != 0:
            PReq.append((A * Jop[i] * Vop[i]) / 1000)  # kW
        else:
            PReq.append(0)

    Degradation_Rate = 0.0015

    def updatePReq(i, Jop_=Jop[i], Vop_=Vop[i]):  # prevents Jop[i] and Vop[i] from being modified

        if Jop[i] != 0:
            PReq[i] = ((A * Jop_ * Vop_) / 1000)  # kW
        else:
            PReq[i] = 0
        return PReq[i]

    Vop_copy = Vop[:]  # stores degraded voltage
    Energy_Charge = {}  
    Degraded_Voltage = {} 
    threshold_list_J = []  # at which current is the threshold reached
    threshold_list_hour = []  # at which hour is threshold reached
    Jop_plotting = []  # to graph the chosen current vs fallback current if threshold reached
    Jop_fallback = []
    Vop_plotting = []  # Vop from the IV curve
    Vop_fallback = []  # Vop fallback that has been degraded based on hour
    for i in range(len(Jop)):  # 90 values, increments of 0.1
        temp_EC = []
        temp_degraded_voltage = [] #testing to accumulate degraded voltage
        count = 0
        current_index = i
        Jop_temp = Jop[i]
        Vop_temp = Vop[i]
        hour = 0  # first hour is 1
        Degrade_count = 0  # counts how many times degradation has been done to apply to the new voltage
        for price in dfElPrice:  
            count += 1
            hour += 1
            if count == 1000:
                Degrade_count += 1
                # print(count)
                count = 0
                if Jop[i] != 0:
                    current_index = i  # voltage currently operating at
                    Vop_copy[i] += Vop_copy[i] * Degradation_Rate  # applies DR every 1k hrs
                    if Vop_copy[i] >= V_threshold:
                        Vop_temp = Vop_copy[i]  # use the degraded voltage
                        temp_EC.append(np.nan)  
                        temp_degraded_voltage.append(Vop_copy[i])
                    else:  # threshold not reached
                        Vop_temp = Vop_copy[i]  # use the degraded voltage
                        temp_EC.append(
                            price * updatePReq(i, Jop[i], Vop_copy[i]))  # calculate Energy Charge with degraded voltage
                        temp_degraded_voltage.append(Vop_copy[i]) # for printing
                    if Jop[
                        current_index] not in threshold_list_J:  # if the threshold occurred, if I already added the Jop DON'T ADD IT AGAIN
                        threshold_list_J.append(Jop[current_index])
                        threshold_list_hour.append(hour)
                    Vop_plotting.append(Vop[i])
                    Vop_fallback.append(Vop_temp)
                    Jop_plotting.append(Jop[i])
                    Jop_fallback.append(Jop[current_index])
                    # print(price*PReq[i])
                else:
                    temp_EC.append(
                        0)  
                    temp_degraded_voltage.append(Vop_copy[i])      
                Energy_Charge["EC_Jop" + str(round(Jop[i], 2))] = temp_EC  # collects Energy Charges in a dictionary
                Degraded_Voltage["DV_Jop" + str(round(Jop[i], 2))] = temp_degraded_voltage
                #print(Vop_copy[i])
            else:  # anything other than thousands (e.g. 1 to 999 and 1001 to 1999, 2001 to 2999)
                if Jop[i] != 0:
                    if Vop_copy[i] >= V_threshold:
                        temp_EC.append(np.nan)
                        temp_degraded_voltage.append(Vop_copy[i])
                    else:
                        temp_EC.append(price * updatePReq(i, Jop[i], Vop_copy[i]))
                        temp_degraded_voltage.append(Vop_copy[i])
                else:
                    temp_EC.append(0)  # EC IS $0 for Jop0
                    temp_degraded_voltage.append(Vop_copy[i])
                Energy_Charge["EC_Jop" + str(round(Jop[i], 2))] = temp_EC
                Degraded_Voltage["DV_Jop" + str(round(Jop[i], 2))] = temp_degraded_voltage

    df_LCOH = pd.DataFrame(data=Energy_Charge)  # holds all 10 years prices
    df_LCOH_total_list = []  # holds all of 10 individual df's, 1 for each year
    start = 0
    stop = Life_1year
    for i in range(10):
        df_LCOH_temp = df_LCOH.iloc[start:stop]
        df_LCOH_total_list.append(df_LCOH_temp)  #saves each year in a list so can loop through
        start = stop
        stop = (i + 2) * Life_1year  #cycles though each year

    for df in df_LCOH_total_list:
        df.index = range(Life_1year)


    
    M_CAPEX = [] # varying the CAPEX_Electronics with Jop
    for i in range(len(Jop)):
        if Jop[i] > 1.7:
            CAPEX_Electronics_Total =  CAPEX_Electronics*A*Jop[i]*Vop[i]/1000
            Installation_Factor = 1.19
            Total_CAPEX = (CAPEX_Rest_Total + CAPEX_Electronics_Total)*Installation_Factor
            M_CAPEX.append(Total_CAPEX/Life_hours) 
        elif Jop[i] > 0 and Jop[i] <= 1.7:
            CAPEX_Electronics_Total =  CAPEX_Electronics*A*Jop_rated*Vop_rated/1000
            Installation_Factor = 1.19
            Total_CAPEX = (CAPEX_Rest_Total + CAPEX_Electronics_Total)*Installation_Factor
            M_CAPEX.append(Total_CAPEX/Life_hours) 
        else:
            M_CAPEX.append(0)  # 0 kg produced when Jop=0
      
    M_OM = [x*0.05*10 for x in M_CAPEX] 
    
    # Rate of fuel production in kg every hour of year 1
    M_kgh2 = []
    for i in range(len(Jop)):
        if Jop[i] != 0:
            M_kgh2.append((Jop[i] * A) / (n_mol * F) * (0.002 * 3600))  # hourly kg H2
        else:
            M_kgh2.append(0)  # 0 kg produced when Jop=0

    for df_LCOH in df_LCOH_total_list:  # cycle through and get each year
        for i in range(len(Jop)):
            df_LCOH["kg_Jop" + str(round(Jop[i], 2))] = [np.nan if np.isnan(df_LCOH["EC_Jop" + str(round(Jop[i], 2))][j]) \
                                                            else M_kgh2[i] for j in range(
                Life_1year)]  # 10.27.20 - list, if EC for that hour and current is NaN, make Nan, else use the calc
            df_LCOH["CAP_Jop" + str(round(Jop[i], 2))] = [np.nan if np.isnan(df_LCOH["EC_Jop" + str(round(Jop[i], 2))][j]) \
                                                              else M_CAPEX[i] for j in range(Life_1year)]
            df_LCOH["OM_Jop" + str(round(Jop[i], 2))] = [np.nan if np.isnan(df_LCOH["EC_Jop" + str(round(Jop[i], 2))][j]) \
                                                            else M_OM[i] for j in range(Life_1year)]
    col1 = []
    for df_LCOH in df_LCOH_total_list:
        for i in range(len(Jop)):  # 10.9.20 - length now 91
            CAP_name = 'CAP_Jop' + str(round(Jop[i], 2))
            OM_name = 'OM_Jop' + str(round(Jop[i], 2))
            EC_name = 'EC_Jop' + str(round(Jop[i], 2))
            kg_name = 'kg_Jop' + str(round(Jop[i], 2))

            col1.append(CAP_name)
            col1.append(OM_name)
            col1.append(EC_name)
            col1.append(kg_name)

        df_LCOH = df_LCOH[col1]


    # COMPARE ALL COLUMNS AND PICK UP THE MIN mlCOH - no comparison for mode 1A
    for df_LCOH in df_LCOH_total_list:  
        for i in range(len(Jop)):
                df_LCOH["mLCOH_Jop" + str(round(Jop[i], 2))] = (df_LCOH["CAP_Jop" + str(round(Jop[i], 2))] + df_LCOH[
                    "OM_Jop" + str(round(Jop[i], 2))] + df_LCOH["EC_Jop" + str(round(Jop[i], 2))]) / df_LCOH[
                                                                  "kg_Jop" + str(round(Jop[i], 2))]
    
        mLCOH_title_list = []

        for i in range(len(Jop)):  # 10.13.20
            mLCOH_title_list.append("mLCOH_Jop" + str(round(Jop[i], 2)))

        df_LCOH["mLCOH_min"] = df_LCOH[mLCOH_title_list].min(axis=1)
       
        df_LCOH[mLCOH_title_list].loc[
            0].min()
        df_LCOH


    # sum the EC associated with the mLCOH for the annnual LCOH
    OptimalEC_List_DCFA = []  # 10.13.20 - for all 10 years
    for df_LCOH in df_LCOH_total_list:
        OptimalEC_Year1 = 0  
        OptimalJop_Year1_List = []
        OptimalEC_Year1_List = []

        for i in range(len(df_LCOH["mLCOH_Jop"+str(round(Jop_,2))])):  # we take length of the year; i is along row - 8760 hours
            mLCOH_list = []
            # mLCOH_list = [df_LCOH["mLCOH_Jop0"][i], df_LCOH["mLCOH_Jop1"][i], df_LCOH["mLCOH_Jop4"][i]]
            for k in range(len(Jop)):  # k collects 91 values into list (instead of 3)
                mLCOH_list.append(df_LCOH["mLCOH_Jop" + str(round(Jop[k], 2))][i])
            temp_mLCOH_min = min(mLCOH_list)
            # print(len(mLCOH_list))

            # then for the minimum LCOH find the location inside the list
            for j in range(len(mLCOH_list)):
                if temp_mLCOH_min == mLCOH_list[j]:  # take the index from the min LCOH
                    index = j  # save the position for which it is associated so it can be used next line
            if i == 0: 
                pass
            OptimalEC_Year1 += df_LCOH["EC_Jop" + str(round(Jop[index], 2))][i]  # sums the corresponding min EC
            OptimalEC_Year1_List.append(df_LCOH["EC_Jop" + str(round(Jop[index], 2))][i])  # adds all to df
            OptimalJop_Year1_List.append(Jop[index])

        OptimalEC_List_DCFA.append(OptimalEC_Year1)  # 10.13.20

        df_LCOH["m_ECmin"] = OptimalEC_Year1_List  # creates the df column
        df_LCOH["OptimalJop"] = OptimalJop_Year1_List

   
    # for the minimum find the corresponding kg, put in a new column and sum
    OptimalKG_List_DCFA = [] 
    for df_LCOH in df_LCOH_total_list:
        OptimalKG_Year1 = 0
        # cycle through each row and find the minimum LCOH
        OptimalKG_Year1_List = []
        for i in range(len(df_LCOH["mLCOH_Jop"+str(round(Jop_,2))])): #2.0
            mLCOH_list = []
            for k in range(len(Jop)):  # k collects 91 values into list (instead of 3)
                mLCOH_list.append(df_LCOH["mLCOH_Jop" + str(round(Jop[k], 2))][i])
            temp_mLCOH_min = min(mLCOH_list)

            # then for the minimum LCOH find the location inside the list
            for j in range(len(mLCOH_list)):
                if temp_mLCOH_min == mLCOH_list[j]:  # take the index from the min LCOH
                    index = j  # save the position for which it is associated so it can be used next line
            if i == 0:  
                pass
            OptimalKG_Year1 += df_LCOH["kg_Jop" + str(round(Jop[index], 2))][i]  # sums the corresponding min kg
            OptimalKG_Year1_List.append(df_LCOH["kg_Jop" + str(round(Jop[index], 2))][i])  # adds all to list to check

        OptimalKG_List_DCFA.append(OptimalKG_Year1)  # 10.13.20

        df_LCOH["m_kgmin"] = OptimalKG_Year1_List

    for df_LCOH in df_LCOH_total_list:  # 10.13.20
        # CAPEX YEAR 1
        CAPEX_Year1 = df_LCOH["CAP_Jop"+str(round(Jop_,2))].sum() #2.0

        # OM YEAR 1
        OM_Year1 = df_LCOH["OM_Jop"+str(round(Jop_,2))].sum()

        # LCOH MIN for year 1
        LCOHmin_Year1 = (CAPEX_Year1 + OM_Year1 + OptimalEC_Year1) / OptimalKG_Year1

    df_DCFA = pd.DataFrame(data={})
    df_DCFA["Year"] = list(range(0, 11))
    
    df_DCFA["CAPEX"] = [Total_CAPEX if i == 0 else 0 for i in range(11)]

    df_DCFA["PWF"] = [1 / (1 + DR) ** df_DCFA["Year"][i] for i in range(11)]

    df_DCFA["PV_OM"] = [0 if i == 0 else (OM_Year1 * df_DCFA["PWF"][i]) for i in range(11)]

    OptimalEC_List_DCFA.insert(0, 0)
    df_DCFA["PV_EC"] = [0 if i == 0 else (OptimalEC_List_DCFA[i] * df_DCFA["PWF"][i]) for i in range(11)]

    OptimalKG_List_DCFA.insert(0, 0)
    df_DCFA["PV_KG"] = [0 if i == 0 else (OptimalKG_List_DCFA[i] * df_DCFA["PWF"][i]) for i in range(11)]

    # Calculate LCOH

    PV_Costs = (df_DCFA["CAPEX"].sum() + df_DCFA["PV_OM"].sum() + df_DCFA[
        "PV_EC"].sum())  

    PV_KG = df_DCFA["PV_KG"].sum()

    Lifetime_LCOH_dynamic_Jop0baseline = PV_Costs / PV_KG

    # CONTRIBUTION ANALYSIS
    CAPEX_Cont = df_DCFA["CAPEX"].sum() / PV_Costs * 100

    OM_Cont = df_DCFA["PV_OM"].sum() / PV_Costs * 100

    EC_Cont = df_DCFA["PV_EC"].sum() / PV_Costs * 100

    df_DCFA["PV_KG"].sum()


    print(Jop_, Lifetime_LCOH_dynamic_Jop0baseline, CAPEX_per_1kW, Total_CAPEX, df_DCFA["PV_EC"].sum(), df_DCFA["PV_OM"].sum(), PV_Costs, PV_KG, file=f_file,sep=",") #1.9.21 - write to csv as 3 columns
    Jop_ += 0.1 #the step interval for Jop_rated; must be down here or order is wrong
    print("UNINST_CAPEXperKW:", CAPEX_per_1kW)
    print("INSTALL_CAPEX:", Total_CAPEX)
    print("PV_EC:", df_DCFA["PV_EC"].sum())
    print("PV_KG:", PV_KG)
    print("PV_OM:", df_DCFA["PV_OM"].sum())
    print("LCOH:", Lifetime_LCOH_dynamic_Jop0baseline)
f_file.close()
df_csv = pd.read_csv("figure1a.csv")
df_csv

Jop_: 1.7 Jop_rated: 1.7 Vop_rated: 1.743136145279
Jop: 1.7
Vop: 1.74313614527871
UNINST_CAPEXperKW: 872
INSTALL_CAPEX: 11597600.0
PV_EC: 19339492.043658596
PV_KG: 11551691.058712926
PV_OM: 3563111.5732560293
LCOH: 2.9865933430493414


,Jop,LCOH,UNINST_CAPEXperKW,INST_CAPEX,OPEX,OM,PV_Costs,PV_KG
0,1.7,2.986593,872,11597600.0,1.933949e+07,3.563112e+06,3.450020e+07,1.155169e+07


# 4.) Mode 1B - Any Jop in range Constantly

In [ ]:
import math  # for the exponential function
import numpy as np
import matplotlib.pyplot as plt

#write results to csv
f_file = open("figure1b.csv","w")
print("Mean Price","Jop","Average Jop","LCOH","INST_CAPEX", "OPEX","OM","PV_Costs", "PV_KG", file=f_file,sep=",")

for mean in df_array_dict: #use to run for all price distributions
    print("Mean Price:", mean)
    
    # Technical Variables
    Jop_rated = 1.7 # A / cm2
    Vop_rated = 1.743136145279 # Volts
    Capacity = 1000  # Conversion factor
    Capacity_kW = 10000 #kW
    Stack_cost_1kW = 473.92 #2020 USD 
    M_BOP_1kW = 150.10 #2020 USD 
    E_BOP_1kW = 130.48 #2020 USD 
    CAPEX_per_1kW = Stack_cost_1kW + M_BOP_1kW + E_BOP_1kW 
    Electronics_percentage = 0.26 #4.5.21 - includes EBOP, phase separator and dryers
    CAPEX_Electronics = Electronics_percentage*CAPEX_per_1kW
    CAPEX_Rest = (1-Electronics_percentage)*CAPEX_per_1kW
    A = (Capacity * Capacity_kW) / (Vop_rated * Jop_rated)  # cm2 electrode area
    CAPEX_Rest_Total = CAPEX_Rest*A*Jop_rated*Vop_rated/1000 #1000 converts to kW

    Jop_ = 0.1 
    Y = lambda X: ((0.156726387554 * X) + 1.47670128643691) #x is Jop_
    while Jop_ <= 6.0: #keep increase at one decimal point increase
        print("Jop_:", Jop_, "Jop_rated:", Jop_rated, "Vop_rated:", Vop_rated)
        Vop_ = Y(Jop_) 

        n_mol = 2  # constant for H2
        F = 96485  # Faraday's constant
        V_threshold = 2.9  # 10.9.20 - voltage at which current removed from options
        
        # Financial Variables
        N_life = 10  # life of electrolyzer
        DR = 0.10  # discount rate

        Jop = [Jop_] 
        print("Jop:",Jop[0])
        
        Vop = [Vop_] 
        print("Vop:",Vop[0])
    

        PReq = []  # kW required per hour
        for i in range(len(Jop)): 
            if Jop[i] != 0:
                PReq.append((A * Jop[i] * Vop[i]) / 1000)  # kW
            else:
                PReq.append(0)

        Degradation_Rate = 0.0015 #0.15%

        def updatePReq(i, Jop_=Jop[i], Vop_=Vop[i]):  # prevents Jop[i] and Vop[i] from being modified

            if Jop[i] != 0:
                PReq[i] = ((A * Jop_ * Vop_) / 1000)  # kW
            else:
                PReq[i] = 0
            return PReq[i]

        Vop_copy = Vop[:]  
        Energy_Charge = {}  
        Degraded_Voltage = {} 
        threshold_list_J = []  
        threshold_list_hour = []   
        Jop_plotting = [] 
        Jop_fallback = []
        Vop_plotting = []  
        Vop_fallback = [] 
        for i in range(len(Jop)): 
            temp_EC = []
            temp_degraded_voltage = [] #testing to accumulate degraded voltage
            count = 0
            current_index = i
            Jop_temp = Jop[i]
            Vop_temp = Vop[i]
            hour = 0  
            Degrade_count = 0  
            for price in df_array_dict[mean]: 
                count += 1
                hour += 1
                if count == 1000:
                    Degrade_count += 1
                    count = 0
                    if Jop[i] != 0:
                        current_index = i  
                        Vop_copy[i] += Vop_copy[i] * Degradation_Rate  # applies deg rate every 1k hrs
                        if Vop_copy[i] >= V_threshold:
                            Vop_temp = Vop_copy[i]  # use the degraded voltage
                            temp_EC.append(np.nan)  # make EC nan
                            temp_degraded_voltage.append(Vop_copy[i])                  
                        else:  # threshold not reached
                            Vop_temp = Vop_copy[i]  # use the degraded voltage
                            temp_EC.append(
                                price * updatePReq(i, Jop[i], Vop_copy[i]))  # calculate Energy Charge with degraded voltage
                            temp_degraded_voltage.append(Vop_copy[i]) #for printing
                        if Jop[
                            current_index] not in threshold_list_J:  
                            threshold_list_J.append(Jop[current_index])
                            threshold_list_hour.append(hour)
                        Vop_plotting.append(Vop[i])
                        Vop_fallback.append(Vop_temp)
                        Jop_plotting.append(Jop[i])
                        Jop_fallback.append(Jop[current_index])
                        # print(price*PReq[i])
                    else:
                        temp_EC.append(
                            0)  
                        temp_degraded_voltage.append(Vop_copy[i])
                    Energy_Charge["EC_Jop" + str(round(Jop[i], 2))] = temp_EC  # collects Energy Charges in a dictionary
                    Degraded_Voltage["DV_Jop" + str(round(Jop[i], 2))] = temp_degraded_voltage
                else:  # anything other than thousands (e.g. 1 to 999 and 1001 to 1999, 2001 t0 2999)
                    if Jop[i] != 0:
                        if Vop_copy[i] >= V_threshold:
                            temp_EC.append(np.nan)
                            temp_degraded_voltage.append(Vop_copy[i])
                        else:
                            temp_EC.append(price * updatePReq(i, Jop[i], Vop_copy[i]))
                            temp_degraded_voltage.append(Vop_copy[i])             
                    else:
                        temp_EC.append(0)  # EC IS $0 for Jop0
                        temp_degraded_voltage.append(Vop_copy[i])
                    Energy_Charge["EC_Jop" + str(round(Jop[i], 2))] = temp_EC
                    Degraded_Voltage["DV_Jop" + str(round(Jop[i], 2))] = temp_degraded_voltage

        df_LCOH = pd.DataFrame(data=Energy_Charge)  
        df_LCOH_total_list = []  
        start = 0
        stop = Life_1year
        for i in range(10):
            df_LCOH_temp = df_LCOH.iloc[start:stop]
            df_LCOH_total_list.append(df_LCOH_temp)  # saves each year in a list so can loop through
            start = stop
            stop = (i + 2) * Life_1year  # cycles though each year

        for df in df_LCOH_total_list:
            df.index = range(Life_1year)


        
        M_CAPEX = [] 
        for i in range(len(Jop)):
            if Jop[i] > 1.7:
                CAPEX_Electronics_Total =  CAPEX_Electronics*A*Jop[i]*Vop[i]/1000
                Installation_Factor = 1.19
                Total_CAPEX = (CAPEX_Rest_Total + CAPEX_Electronics_Total)*Installation_Factor
                M_CAPEX.append(Total_CAPEX/Life_hours) 
            elif Jop[i] > 0 and Jop[i] <= 1.7:
                CAPEX_Electronics_Total =  CAPEX_Electronics*A*Jop_rated*Vop_rated/1000
                Installation_Factor = 1.19
                Total_CAPEX = (CAPEX_Rest_Total + CAPEX_Electronics_Total)*Installation_Factor
                M_CAPEX.append(Total_CAPEX/Life_hours) 
            else:
                M_CAPEX.append(0)  
          
        M_OM = [x*0.05*10 for x in M_CAPEX] 
        
        # Rate of fuel production in kg every hour of year 1

        M_kgh2 = []
        for i in range(len(Jop)):
            if Jop[i] != 0:
                M_kgh2.append((Jop[i] * A) / (n_mol * F) * (0.002 * 3600))  # hourly kg H2
            else:
                M_kgh2.append(0)  # 0 kg produced when Jop=0

        for df_LCOH in df_LCOH_total_list: # cycle through and get each year
            for i in range(len(Jop)):
                df_LCOH["kg_Jop" + str(round(Jop[i], 2))] = [np.nan if np.isnan(df_LCOH["EC_Jop" + str(round(Jop[i], 2))][j]) \
                                                                else M_kgh2[i] for j in range(
                    Life_1year)]  # 10.27.20 - list, if EC for that hour and current is NaN, make Nan, else use the calc
                df_LCOH["CAP_Jop" + str(round(Jop[i], 2))] = [np.nan if np.isnan(df_LCOH["EC_Jop" + str(round(Jop[i], 2))][j]) \
                                                                  else M_CAPEX[i] for j in range(Life_1year)]
                df_LCOH["OM_Jop" + str(round(Jop[i], 2))] = [np.nan if np.isnan(df_LCOH["EC_Jop" + str(round(Jop[i], 2))][j]) \
                                                                else M_OM[i] for j in range(Life_1year)]
        col1 = []
        for df_LCOH in df_LCOH_total_list:
            for i in range(len(Jop)):  # 10.9.20 - length now 91
                CAP_name = 'CAP_Jop' + str(round(Jop[i], 2))
                OM_name = 'OM_Jop' + str(round(Jop[i], 2))
                EC_name = 'EC_Jop' + str(round(Jop[i], 2))
                kg_name = 'kg_Jop' + str(round(Jop[i], 2))

                col1.append(CAP_name)
                col1.append(OM_name)
                col1.append(EC_name)
                col1.append(kg_name)

            df_LCOH = df_LCOH[col1]


        # COMPARE ALL COLUMNS AND PICK UP THE MIN mlCOH - no comparison in mode 1B
        for df_LCOH in df_LCOH_total_list:  
            for i in range(len(Jop)):
                    df_LCOH["mLCOH_Jop" + str(round(Jop[i], 2))] = (df_LCOH["CAP_Jop" + str(round(Jop[i], 2))] + df_LCOH[
                        "OM_Jop" + str(round(Jop[i], 2))] + df_LCOH["EC_Jop" + str(round(Jop[i], 2))]) / df_LCOH[
                                                                      "kg_Jop" + str(round(Jop[i], 2))]
            mLCOH_title_list = []

            for i in range(len(Jop)):  # 10.13.20
                mLCOH_title_list.append("mLCOH_Jop" + str(round(Jop[i], 2)))

            df_LCOH["mLCOH_min"] = df_LCOH[mLCOH_title_list].min(axis=1)
            
            df_LCOH[mLCOH_title_list].loc[
                0].min()
            df_LCOH


        # sum the EC associated with the mLCOH for the annnual LCOH
        OptimalEC_List_DCFA = []  #for all 10 years
        for df_LCOH in df_LCOH_total_list:
            OptimalEC_Year1 = 0  
            OptimalJop_Year1_List = []
            OptimalEC_Year1_List = []

            for i in range(len(df_LCOH["mLCOH_Jop"+str(round(Jop_,2))])):  
                mLCOH_list = []
                for k in range(len(Jop)):  
                    mLCOH_list.append(df_LCOH["mLCOH_Jop" + str(round(Jop[k], 2))][i])
                temp_mLCOH_min = min(mLCOH_list)
                
                for j in range(len(mLCOH_list)):
                    if temp_mLCOH_min == mLCOH_list[j]:  # take the index from the min LCOH
                        index = j  # save the position for which it is associated so it can be used next line
                if i == 0:  
                    pass
                OptimalEC_Year1 += df_LCOH["EC_Jop" + str(round(Jop[index], 2))][i]  # sums the corresponding min EC
                OptimalEC_Year1_List.append(df_LCOH["EC_Jop" + str(round(Jop[index], 2))][i])  # adds all to df
                OptimalJop_Year1_List.append(Jop[index])

            OptimalEC_List_DCFA.append(OptimalEC_Year1)  

            df_LCOH["m_ECmin"] = OptimalEC_Year1_List  # creates the df column
            df_LCOH["OptimalJop"] = OptimalJop_Year1_List

        # for the minimum find the corresponding kg, put in a new column and sum
        OptimalKG_List_DCFA = []  
        for df_LCOH in df_LCOH_total_list:
            OptimalKG_Year1 = 0
            # cycle through each row and find the minimum LCOH
            OptimalKG_Year1_List = []
            for i in range(len(df_LCOH["mLCOH_Jop"+str(round(Jop_,2))])): #2.0
                mLCOH_list = []
                for k in range(len(Jop)):  
                    mLCOH_list.append(df_LCOH["mLCOH_Jop" + str(round(Jop[k], 2))][i])
                temp_mLCOH_min = min(mLCOH_list)

                # then for the minimum LCOH find the location inside the list
                for j in range(len(mLCOH_list)):
                    if temp_mLCOH_min == mLCOH_list[j]:  # take the index from the min LCOH
                        index = j  # save the position for which it is associated so it can be used next line
                if i == 0:  
                    pass
                OptimalKG_Year1 += df_LCOH["kg_Jop" + str(round(Jop[index], 2))][i]  # sums the corresponding min kg
                OptimalKG_Year1_List.append(df_LCOH["kg_Jop" + str(round(Jop[index], 2))][i])  # adds all to list to check

            OptimalKG_List_DCFA.append(OptimalKG_Year1)  

            df_LCOH["m_kgmin"] = OptimalKG_Year1_List

        for df_LCOH in df_LCOH_total_list:  
            # CAPEX YEAR 1
            CAPEX_Year1 = df_LCOH["CAP_Jop"+str(round(Jop_,2))].sum() 

            # OM YEAR 1
            OM_Year1 = df_LCOH["OM_Jop"+str(round(Jop_,2))].sum()

            # LCOH MIN for year 1
            LCOHmin_Year1 = (CAPEX_Year1 + OM_Year1 + OptimalEC_Year1) / OptimalKG_Year1

        df_DCFA = pd.DataFrame(data={})
        df_DCFA["Year"] = list(range(0, 11))
        
        df_DCFA["CAPEX"] = [Total_CAPEX if i == 0 else 0 for i in range(11)]
        # df_DCFA

        df_DCFA["PWF"] = [1 / (1 + DR) ** df_DCFA["Year"][i] for i in range(11)]

        # OM
        df_DCFA["PV_OM"] = [0 if i == 0 else (OM_Year1 * df_DCFA["PWF"][i]) for i in range(11)]

        # EC
        OptimalEC_List_DCFA.insert(0, 0)
        df_DCFA["PV_EC"] = [0 if i == 0 else (OptimalEC_List_DCFA[i] * df_DCFA["PWF"][i]) for i in range(11)]

        # kg
        OptimalKG_List_DCFA.insert(0, 0)
        df_DCFA["PV_KG"] = [0 if i == 0 else (OptimalKG_List_DCFA[i] * df_DCFA["PWF"][i]) for i in range(11)]

        # Calculate LCOH

        PV_Costs = (df_DCFA["CAPEX"].sum() + df_DCFA["PV_OM"].sum() + df_DCFA[
            "PV_EC"].sum())  
        
        PV_KG = df_DCFA["PV_KG"].sum()

        Lifetime_LCOH_dynamic_Jop0baseline = PV_Costs / PV_KG

        # CONTRIBUTION ANALYSIS
        CAPEX_Cont = df_DCFA["CAPEX"].sum() / PV_Costs * 100

        OM_Cont = df_DCFA["PV_OM"].sum() / PV_Costs * 100

        EC_Cont = df_DCFA["PV_EC"].sum() / PV_Costs * 100

        df_DCFA["PV_KG"].sum()


        print(mean,Jop_, df_LCOH["OptimalJop"].mean(),Lifetime_LCOH_dynamic_Jop0baseline, df_DCFA["CAPEX"].sum(), df_DCFA["PV_EC"].sum(), df_DCFA["PV_OM"].sum(), PV_Costs, PV_KG, file=f_file,sep=",") #1.9.21 - write to csv as 3 columns
        Jop_ += 0.1 #the step interval for Jop_rated
        print("Average Jop:", df_LCOH["OptimalJop"].mean())
        print("UNINST_CAPEXperKW:", CAPEX_per_1kW)
        print("INST_CAPEX:", df_DCFA["CAPEX"].sum())
        print("PV_EC:", df_DCFA["PV_EC"].sum())
        print("PV_KG:", PV_KG)
        print("PV_OM:", df_DCFA["PV_OM"].sum())
        print("LCOH:", Lifetime_LCOH_dynamic_Jop0baseline)
f_file.close()
df_csv = pd.read_csv("figure1b.csv")
df_csv

Mean Price: 0.0
Jop_: 0.1 Jop_rated: 1.7 Vop_rated: 1.743136145279
Jop: 0.1
Vop: 1.49237392519231
Average Jop: 0.10000000000001491
UNINST_CAPEXperKW: 754.5
INST_CAPEX: 8978550.0
PV_EC: 678.648137648419
PV_KG: 677731.8059241057
PV_OM: 2758465.149346238
LCOH: 17.31908358864643
Jop_: 0.2 Jop_rated: 1.7 Vop_rated: 1.743136145279
Jop: 0.2
Vop: 1.50804656394771
Average Jop: 0.20000000000002982
UNINST_CAPEXperKW: 754.5
INST_CAPEX: 8978550.0
PV_EC: 1371.550353210882
PV_KG: 1355463.6118482114
PV_OM: 2758465.149346238
LCOH: 8.66005298636814
Jop_: 0.30000000000000004 Jop_rated: 1.7 Vop_rated: 1.743136145279
Jop: 0.30000000000000004
Vop: 1.5237192027031101
Average Jop: 0.30000000000001514
UNINST_CAPEXperKW: 754.5
INST_CAPEX: 8978550.0
PV_EC: 2078.7066466875785
PV_KG: 2033195.417772461
PV_OM: 2758465.149346238
LCOH: 5.773716462952737
Jop_: 0.4 Jop_rated: 1.7 Vop_rated: 1.743136145279
Jop: 0.4
Vop: 1.53939184145851
Average Jop: 0.40000000000005964
UNINST_CAPEXperKW: 754.5
INST_CAPEX: 8978550.0
PV_EC

,Mean Price,Jop,Average Jop,LCOH,INST_CAPEX,OPEX,OM,PV_Costs,PV_KG
0,0.00,0.1,0.1,17.319084,8.978550e+06,6.786481e+02,2.758465e+06,1.173769e+07,6.777318e+05
1,0.00,0.2,0.2,8.660053,8.978550e+06,1.371550e+03,2.758465e+06,1.173839e+07,1.355464e+06
2,0.00,0.3,0.3,5.773716,8.978550e+06,2.078707e+03,2.758465e+06,1.173909e+07,2.033195e+06
3,0.00,0.4,0.4,4.330553,8.978550e+06,2.800117e+03,2.758465e+06,1.173982e+07,2.710927e+06
4,0.00,0.5,0.5,3.464660,8.978550e+06,3.535781e+03,2.758465e+06,1.174055e+07,3.388659e+06
...,...,...,...,...,...,...,...,...,...
475,0.07,5.6,5.6,5.256917,1.703045e+07,1.772530e+08,5.232238e+06,1.995157e+08,3.795298e+07
476,0.07,5.7,5.7,5.286373,1.728630e+07,1.816192e+08,5.310840e+06,2.042164e+08,3.863071e+07
477,0.07,5.8,5.8,5.315967,1.754461e+07,1.860276e+08,5.390202e+06,2.089624e+08,3.930844e+07
478,0.07,5.9,5.9,5.345693,1.780539e+07,1.904781e+08,5.470322e+06,2.137538e+08,3.998618e+07


# 5.) Mode 2A - Jop of 0.1 OR 1.7

In [ ]:
import math  # for the exponential function
import numpy as np
import matplotlib.pyplot as plt

f_file = open("figure2a.csv","w")
print("Mean Price","Average Jop 10 Yr","LCOH","CAPEX", "OPEX","OM","PV_Costs", "PV_KG", file=f_file,sep=",")

for mean in df_array_dict:
    print("Mean Price:", mean)

    # Technical Variables
    Capacity = 1000  # conversion factor
    Capacity_kW = 10000
    Stack_cost_1kW = 473.92 #2020 USD 
    M_BOP_1kW = 150.10 #2020 USD 
    E_BOP_1kW = 130.48 #2020 USD 
    CAPEX_per_1kW = Stack_cost_1kW + M_BOP_1kW + E_BOP_1kW 
    Installation_Factor = 1.19
    Installed_CAPEX_per_1kW = CAPEX_per_1kW*Installation_Factor
        
    Jop_rated = 1.7 #A / cm2
    Vop_rated = 1.743136145279 #Volts

    A = (Capacity * Capacity_kW) / (Vop_rated * Jop_rated)  
    n_mol = 2  # constant for H2
    F = 96485  # Faraday's constant
    V_threshold = 2.9  # voltage at which current removed from options

    # Financial Variables
    N_life = 10  # 10 years
    CAPEX = (Installed_CAPEX_per_1kW * 10000)  # $/kW x 10,000 kW
    OM = (CAPEX * 0.05)
    DR = 0.10  # discount rate

    M_CAPEX = CAPEX / Life_hours  

    # OM per hour
    M_OM = OM / Life_1year  

    # Selects between 0.1 or 1.7. Cannot select 0, code prevents it

    Jop = [0,0.1,1.7] 
   
    # Corresponding voltages for Jops

    Vop = [1.47670128644,1.492373925192,1.743136145279]

    PReq = []  # kW required per hour
    for i in range(len(Jop)):  
        if Jop[i] != 0:
            PReq.append((A * Jop[i] * Vop[i]) / 1000)  # kW
        else:
            PReq.append(0)
  
    Degradation_Rate = 0.0015 #0.15%

    def updatePReq(i, Jop_=Jop[i], Vop_=Vop[i]):  
        if Jop[i] != 0:
            PReq[i] = ((A * Jop_ * Vop_) / 1000)  # kW
        else:
            PReq[i] = 0
        return PReq[i]

    Vop_copy = Vop[:]  
    Energy_Charge = {}  
    PReq_dict = {}
    Degraded_Voltage = {} 
    threshold_list_J = []  
    threshold_list_hour = [] 
    Jop_plotting = []  
    Jop_fallback = []
    Vop_plotting = []  
    Vop_fallback = []  
    for i in range(len(Jop)):  
        temp_EC = []
        temp_PReq = []
        temp_degraded_voltage = [] 
        count = 0
        current_index = i
        Jop_temp = Jop[i]
        Vop_temp = Vop[i]
        hour = 0  
        Degrade_count = 0  
        for price in df_array_dict[mean]:  
            count += 1
            hour += 1
            if count == 1000:
                Degrade_count += 1
                # print(count)
                count = 0
                if Jop[i] != 0:
                    current_index = i  
                    Vop_copy[i] += Vop_copy[i] * Degradation_Rate  # applies deg rate every 1k hrs
                    if Vop_copy[i] >= V_threshold:
                        Vop_temp = Vop_copy[i]  # use the degraded voltage
                        temp_EC.append(np.nan)  # make EC nan
                        temp_PReq.append(np.nan)
                        temp_degraded_voltage.append(Vop_copy[i])
                    else:  # threshold not reached
                        Vop_temp = Vop_copy[i]  # use the degraded voltage
                        temp_EC.append(
                            price * updatePReq(i, Jop[i], Vop_copy[i]))  # calculate Energy Charge with degraded voltage
                        temp_PReq.append(updatePReq(i, Jop[i], Vop_copy[i]))
                        temp_degraded_voltage.append(Vop_copy[i]) #for printing
                    if Jop[
                        current_index] not in threshold_list_J:  # if the threshold occurred, if I already added the Jop DON'T ADD IT AGAIN
                        threshold_list_J.append(Jop[current_index])
                        threshold_list_hour.append(hour)
                    #                   ,Vop_copy[i],Vop_temp) #Vop[i] is from IV curve, Vop_copy[i] is degraded and Vop_temp is fallback Vop and degraded
                    Vop_plotting.append(Vop[i])
                    Vop_fallback.append(Vop_temp)
                    Jop_plotting.append(Jop[i])
                    Jop_fallback.append(Jop[current_index])
                    # print(price*PReq[i])
                else:
                    temp_EC.append(
                        0)  # EC IS $0 #add degradation rate #Jop0 is a static number for when electrolyzer is off
                    # print(0)
                    temp_PReq.append(0)
                    temp_degraded_voltage.append(Vop_copy[i])
                Energy_Charge["EC_Jop" + str(round(Jop[i], 2))] = temp_EC  # collects Energy Charges in a dictionary
                Degraded_Voltage["DV_Jop" + str(round(Jop[i], 2))] = temp_degraded_voltage
                #print(Vop_copy[i])
            else:  # anything other than thousands (e.g. 1 to 999 and 1001 to 1999, 2001 t0 2999)
                if Jop[i] != 0:
                    if Vop_copy[i] >= V_threshold:
                        temp_EC.append(np.nan)
                        temp_PReq.append(np.nan)
                        temp_degraded_voltage.append(Vop_copy[i])
                    else:
                        temp_EC.append(price * updatePReq(i, Jop[i], Vop_copy[i]))
                        temp_PReq.append(updatePReq(i, Jop[i], Vop_copy[i]))
                        temp_degraded_voltage.append(Vop_copy[i])
                #                
                else:
                    temp_EC.append(0)  # EC IS $0 for Jop0
                    temp_PReq.append(0)
                    temp_degraded_voltage.append(Vop_copy[i])
                Energy_Charge["EC_Jop" + str(round(Jop[i], 2))] = temp_EC
                PReq_dict["PReq_Jop"+ str(round(Jop[i], 2))] = temp_PReq
                Degraded_Voltage["DV_Jop" + str(round(Jop[i], 2))] = temp_degraded_voltage

    df_LCOH = pd.DataFrame(data=Energy_Charge)  # 10.10.20 - holds all 87,600 ECs
    for key in PReq_dict.keys():
        df_LCOH[key]=PReq_dict[key]
    df_LCOH_total_list = []  # 10.10.20 - holds all of 10 individual df's, 1 for each year
    start = 0
    stop = Life_1year
    for i in range(10):
        # print(start,stop)
        df_LCOH_temp = df_LCOH.iloc[start:stop]
        df_LCOH_total_list.append(df_LCOH_temp)  # 10.10.20 - saves each year in a list so can loop through
        start = stop
        stop = (i + 2) * Life_1year  # 10.10 - cycles though each year
    for df in df_LCOH_total_list:
        df.index = range(Life_1year)
  
    # Rate of fuel production in kg every hour 
    M_kgh2 = []
    for i in range(len(Jop)):
        if Jop[i] != 0:
            M_kgh2.append((Jop[i] * A) / (n_mol * F) * (0.002 * 3600))  # hourly kg H2
        else:
            M_kgh2.append(0)  # 0 kg produced when Jop=0

    for df_LCOH in df_LCOH_total_list:  # cycle through and get each year
        for i in range(len(Jop)):
            df_LCOH["kg_Jop" + str(round(Jop[i], 2))] = [np.nan if np.isnan(df_LCOH["EC_Jop" + str(round(Jop[i], 2))][j]) \
                                                            else M_kgh2[i] for j in range(
                Life_1year)]  # 10.27.20 - list, if EC for that hour and current is NaN, make Nan, else use the calc
            df_LCOH["CAP_Jop" + str(round(Jop[i], 2))] = [np.nan if np.isnan(df_LCOH["EC_Jop" + str(round(Jop[i], 2))][j]) \
                                                              else M_CAPEX for j in range(Life_1year)]
            df_LCOH["OM_Jop" + str(round(Jop[i], 2))] = [np.nan if np.isnan(df_LCOH["EC_Jop" + str(round(Jop[i], 2))][j]) \
                                                            else M_OM for j in range(Life_1year)]
    col1 = []
    for df_LCOH in df_LCOH_total_list:
        for i in range(len(Jop)):  #2.3.21 - 10.9.20 - length now 91
            CAP_name = 'CAP_Jop' + str(round(Jop[i], 2))
            OM_name = 'OM_Jop' + str(round(Jop[i], 2))
            EC_name = 'EC_Jop' + str(round(Jop[i], 2))
            kg_name = 'kg_Jop' + str(round(Jop[i], 2))

            col1.append(CAP_name)
            col1.append(OM_name)
            col1.append(EC_name)
            col1.append(kg_name)

        df_LCOH = df_LCOH[col1]


    # COMPARE ALL COLUMNS AND PICK UP THE MIN mlCOH
    for df_LCOH in df_LCOH_total_list:  # 10.13.20
        for i in range(len(Jop)):
            if i != 0:
                df_LCOH["mLCOH_Jop" + str(round(Jop[i], 2))] = (df_LCOH["CAP_Jop" + str(round(Jop[i], 2))] + df_LCOH[
                    "OM_Jop" + str(round(Jop[i], 2))] + df_LCOH["EC_Jop" + str(round(Jop[i], 2))]) / df_LCOH[
                                                                  "kg_Jop" + str(round(Jop[i], 2))]
            else:
                df_LCOH["mLCOH_Jop" + str(round(Jop[i], 2))] = 10000  # zero never selected

        mLCOH_title_list = []

        for i in range(len(Jop)):  # 2.3.21
            mLCOH_title_list.append("mLCOH_Jop" + str(round(Jop[i], 2)))

        df_LCOH["mLCOH_min"] = df_LCOH[mLCOH_title_list].min(axis=1)
        
        df_LCOH[mLCOH_title_list].loc[
            0].min()
        df_LCOH
  
    # sum the EC associated with the mLCOH for the annnual LCOH
    OptimalEC_List_DCFA = []  # for all 10 years
    for df_LCOH in df_LCOH_total_list:
        OptimalEC_Year1 = 0  # makes for EACH year new columns
        OptimalJop_Year1_List = []
        OptimalEC_Year1_List = []

        for i in range(len(df_LCOH["mLCOH_Jop0.1"])): #take length of the year; i is along row (60 values of Jop)
            mLCOH_list = []
            # mLCOH_list = [df_LCOH["mLCOH_Jop0"][i], df_LCOH["mLCOH_Jop1"][i], df_LCOH["mLCOH_Jop4"][i]]
            for k in range(len(Jop)):  
                mLCOH_list.append(df_LCOH["mLCOH_Jop" + str(round(Jop[k], 2))][i])
            temp_mLCOH_min = min(mLCOH_list)

            # then for the minimum LCOH find the location inside the list
            for j in range(len(mLCOH_list)):
                if temp_mLCOH_min == mLCOH_list[j]:  # take the index from the min LCOH
                    index = j  # save the position for which it is associated so it can be used next line
            if i == 0:  
                pass
            OptimalEC_Year1 += df_LCOH["EC_Jop" + str(round(Jop[index], 2))][i]  # sums the corresponding min EC
            OptimalEC_Year1_List.append(df_LCOH["EC_Jop" + str(round(Jop[index], 2))][i])  # adds all to df
            OptimalJop_Year1_List.append(Jop[index])

        OptimalEC_List_DCFA.append(OptimalEC_Year1)  

        df_LCOH["m_ECmin"] = OptimalEC_Year1_List  # creates the df column
        df_LCOH["OptimalJop"] = OptimalJop_Year1_List

    # for the minimum find the corresponding kg, put in a new column and sum
    OptimalKG_List_DCFA = []  
    for df_LCOH in df_LCOH_total_list:
        OptimalKG_Year1 = 0
        # cycle through each row and find the minimum LCOH
        OptimalKG_Year1_List = []
        for i in range(len(df_LCOH["mLCOH_Jop0.1"])): #2.3.21
            mLCOH_list = []
            # mLCOH_list = [df_LCOH["mLCOH_Jop0"][i], df_LCOH["mLCOH_Jop1"][i], df_LCOH["mLCOH_Jop4"][i]]
            for k in range(len(Jop)):  #2.3.21 k collects 91 values into list (instead of 3)
                mLCOH_list.append(df_LCOH["mLCOH_Jop" + str(round(Jop[k], 2))][i])
            temp_mLCOH_min = min(mLCOH_list)

            # then for the minimum LCOH find the location inside the list
            for j in range(len(mLCOH_list)):
                if temp_mLCOH_min == mLCOH_list[j]:  # take the index from the min LCOH
                    index = j  # save the position for which it is associated so it can be used next line
            if i == 0:  # FOR TESTING
                # print(index)
                pass
            OptimalKG_Year1 += df_LCOH["kg_Jop" + str(round(Jop[index], 2))][i]  # sums the corresponding min kg
            OptimalKG_Year1_List.append(df_LCOH["kg_Jop" + str(round(Jop[index], 2))][i])  # adds all to list to check

        OptimalKG_List_DCFA.append(OptimalKG_Year1)  # 10.13.20

        df_LCOH["m_kgmin"] = OptimalKG_Year1_List

    OptimalPReq_List_DCFA = []  # 10.13.20
    for df_LCOH in df_LCOH_total_list:
        OptimalPReq_Year1 = 0
        # cycle through each row and find the minimum LCOH
        OptimalPReq_Year1_List = []
        for i in range(len(df_LCOH["mLCOH_Jop0.1"])): #2.3.21
            mLCOH_list = []
            # mLCOH_list = [df_LCOH["mLCOH_Jop0"][i], df_LCOH["mLCOH_Jop1"][i], df_LCOH["mLCOH_Jop4"][i]]
            for k in range(len(Jop)):  #2.3.21 k collects 91 values into list (instead of 3)
                mLCOH_list.append(df_LCOH["mLCOH_Jop" + str(round(Jop[k], 2))][i])
            temp_mLCOH_min = min(mLCOH_list)

            # then for the minimum LCOH find the location inside the list
            for j in range(len(mLCOH_list)):
                if temp_mLCOH_min == mLCOH_list[j]:  # take the index from the min LCOH
                    index = j  # save the position for which it is associated so it can be used next line
            if i == 0:  # FOR TESTING
                # print(index)
                pass
            OptimalPReq_Year1 += df_LCOH["PReq_Jop" + str(round(Jop[index], 2))][i]  # sums the corresponding min kg
            OptimalPReq_Year1_List.append(df_LCOH["PReq_Jop" + str(round(Jop[index], 2))][i])  # adds all to list to check

        OptimalPReq_List_DCFA.append(OptimalPReq_Year1)  

        df_LCOH["m_PReqmin"] = OptimalPReq_Year1_List

    for df_LCOH in df_LCOH_total_list:  
        # CAPEX YEAR 1
        CAPEX_Year1 = df_LCOH["CAP_Jop0.1"].sum()

        # OM YEAR 1
        OM_Year1 = df_LCOH["OM_Jop0.1"].sum()

        # LCOH MIN for year 1
        LCOHmin_Year1 = (CAPEX_Year1 + OM_Year1 + OptimalEC_Year1) / OptimalKG_Year1

  
    df_DCFA = pd.DataFrame(data={})
    df_DCFA["Year"] = list(range(0, 11))

    df_DCFA["CAPEX"] = [CAPEX if i == 0 else 0 for i in range(11)]

    df_DCFA["PWF"] = [1 / (1 + DR) ** df_DCFA["Year"][i] for i in range(11)]

    df_DCFA["PV_OM"] = [0 if i == 0 else (OM_Year1 * df_DCFA["PWF"][i]) for i in range(11)]

    OptimalEC_List_DCFA.insert(0, 0)
    df_DCFA["PV_EC"] = [0 if i == 0 else (OptimalEC_List_DCFA[i] * df_DCFA["PWF"][i]) for i in range(11)]

    OptimalKG_List_DCFA.insert(0, 0)
    df_DCFA["PV_KG"] = [0 if i == 0 else (OptimalKG_List_DCFA[i] * df_DCFA["PWF"][i]) for i in range(11)]

    # Calculate LCOH

    PV_Costs = (df_DCFA["CAPEX"].sum() + df_DCFA["PV_OM"].sum() + df_DCFA[
        "PV_EC"].sum())  

    PV_KG = df_DCFA["PV_KG"].sum()

    Lifetime_LCOH_dynamic_Jop0baseline = PV_Costs / PV_KG

    # CONTRIBUTION ANALYSIS
    CAPEX_Cont = df_DCFA["CAPEX"].sum() / PV_Costs * 100

    OM_Cont = df_DCFA["PV_OM"].sum() / PV_Costs * 100

    EC_Cont = df_DCFA["PV_EC"].sum() / PV_Costs * 100

    Lifetime_LCOH_dynamic_Jop0baseline, CAPEX_Cont, OM_Cont, EC_Cont
   
    # FIND KG OF PRODUCTION
    df_DCFA["PV_KG"].sum()

   # find average Jop over 10 years
    OptimalJop_mean = []
    for year in range(10):
        OptimalJop_mean.append(df_LCOH_total_list[year]["OptimalJop"].mean())
    Average_Jop_10yr = sum(OptimalJop_mean) / len(OptimalJop_mean) 
    Year1_Jop = OptimalJop_mean[0]
    Year10_Jop = OptimalJop_mean[9]

    print("Average Jop 10 Yr:", Average_Jop_10yr)
    print("CAPEX:", df_DCFA["CAPEX"].sum())
    print("PV_OM:", df_DCFA["PV_OM"].sum())
    print("PV_KG:" , df_DCFA["PV_KG"].sum())
    print("PV_EC:", df_DCFA["PV_EC"].sum())
    print("LCOH:", Lifetime_LCOH_dynamic_Jop0baseline)

    print(mean,Average_Jop_10yr,Lifetime_LCOH_dynamic_Jop0baseline, df_DCFA["CAPEX"].sum(), df_DCFA["PV_EC"].sum(), df_DCFA["PV_OM"].sum(), PV_Costs, PV_KG, file=f_file,sep=",") #1.9.21 - write to csv as 3 columns
f_file.close()
df_csv = pd.read_csv("figure2a.csv")

Mean Price: 0.0
Average Jop 10 Yr: 1.695926940639525
CAPEX: 8978549.999999998
PV_OM: 2758465.149346238
PV_KG: 11493950.552781628
PV_EC: -3885364.2670833976
LCOH: 0.6831115938951622
Mean Price: 0.01
Average Jop 10 Yr: 1.695926940639525
CAPEX: 8978549.999999998
PV_OM: 2758465.149346238
PV_KG: 11493950.552781628
PV_EC: 1790302.1394031364
LCOH: 1.1769075590355358
Mean Price: 0.02
Average Jop 10 Yr: 1.69589041095916
CAPEX: 8978549.999999998
PV_OM: 2758465.149346238
PV_KG: 11493769.50305459
PV_EC: 7451336.227713012
LCOH: 1.669456775861021
Mean Price: 0.03
Average Jop 10 Yr: 1.69589041095916
CAPEX: 8978549.999999998
PV_OM: 2758465.149346238
PV_KG: 11493769.50305459
PV_EC: 13126907.951466916
LCOH: 2.1632522815256823
Mean Price: 0.04
Average Jop 10 Yr: 1.6958356164386117
CAPEX: 8978549.999999998
PV_OM: 2758465.149346238
PV_KG: 11493492.677944278
PV_EC: 18780097.770430464
LCOH: 2.6551644286804366
Mean Price: 0.05
Average Jop 10 Yr: 1.6958356164386117
CAPEX: 8978549.999999998
PV_OM: 2758465.14934

# 6.) Mode 2B - Jop of 0.1 OR any between 0.1 and 6.0

In [ ]:
import math  # for the exponential function
import numpy as np
import matplotlib.pyplot as plt


f_file = open("figure2b.csv","w")
print("Mean Price","Second Current","Average Jop 10 Yr","LCOH","INST_CAPEX", "OPEX","OM","PV_Costs", "PV_KG", file=f_file,sep=",")

for mean in df_array_dict:
    print("Mean Price:", mean)
    # Technical Variables
    Capacity = 1000  # watts
    Capacity_kW = 10000
    Stack_cost_1kW = 473.92 #2020 USD 
    M_BOP_1kW = 150.10 #2020 USD 
    E_BOP_1kW = 130.48 #2020 USD
    CAPEX_per_1kW = Stack_cost_1kW + M_BOP_1kW + E_BOP_1kW
    
    Jop_rated = 1.7
    Vop_rated = 1.743136145279
    Electronics_percentage = 0.26 
    CAPEX_Electronics = Electronics_percentage*CAPEX_per_1kW
    CAPEX_Rest = (1-Electronics_percentage)*CAPEX_per_1kW
    A = (Capacity * Capacity_kW) / (Vop_rated * Jop_rated)  
    CAPEX_Rest_Total = CAPEX_Rest*A*Jop_rated*Vop_rated/1000 

    A = (Capacity * Capacity_kW) / (Vop_rated * Jop_rated)  
    n_mol = 2  # constant for H2
    F = 96485  # Faraday's constant
    V_threshold = 2.9  #voltage at which current removed from options

    # Financial Variables
    N_life = 10  # life of electrolyzer
    DR = 0.10  # discount rate


    # EITHER 0.1 OR a second curent between 0.1 and 6.0
    pairwise_results = []
    second_current_start = 0.0
    second_current_list = []
    for i in range(61):  # 10.9.20 - increase to 9A/cm2
        second_current_list.append(second_current_start)
        second_current_start += 0.1
    for second_current in second_current_list[1:]: #starts at 0.1 with 1: (17:18 is 1.7Acm2)
        print("second current:", second_current)
        Jop = [0,0.1,second_current] 

        Vop = [1.47670128644,1.492373925192,(0.156726387554 * (second_current) + 1.47670128643691)] 
    
        PReq = []  # kW required per hour
        for i in range(len(Jop)): 
            if Jop[i] != 0:
                PReq.append((A * Jop[i] * Vop[i]) / 1000)  # kW
            else:
                PReq.append(0)
    
        Degradation_Rate = 0.0015


        def updatePReq(i, Jop_=Jop[i], Vop_=Vop[i]):  # prevents Jop[i] and Vop[i] from being modified
            if Jop[i] != 0:
                PReq[i] = ((A * Jop_ * Vop_) / 1000)  # kW
            else:
                PReq[i] = 0
            return PReq[i]

        Vop_copy = Vop[:]  # STORES DEGRADED VOLTAGE
        Energy_Charge = {}  
        Degraded_Voltage = {}
        threshold_list_J = []  # at which current is the threshold reached
        threshold_list_hour = []  # at which hour is threshold reached
        Jop_plotting = []  # to graph the chosen current vs fallback current if threshold reached
        Jop_fallback = []
        Vop_plotting = []  # Vop from the IV curve
        Vop_fallback = []  # Vop fallback that has been degraded based on hour
        for i in range(len(Jop)):  # 60 values, increments of 0.1
            temp_EC = []
            temp_degraded_voltage = [] #testing to accumulate degraded voltage
            count = 0
            # 10.7.20 - Where we cycle through hours
            current_index = i
            Jop_temp = Jop[i]
            Vop_temp = Vop[i]
            hour = 0  # first hour is 1
            Degrade_count = 0  # counts how many times degradation has been done to apply to the new voltage
            for price in df_array_dict[mean]:  
                count += 1
                hour += 1
                if count == 1000:
                    Degrade_count += 1
                    # print(count)
                    count = 0
                    if Jop[i] != 0
                        current_index = i  
                        Vop_copy[i] += Vop_copy[i] * Degradation_Rate  
                        if Vop_copy[i] >= V_threshold:
                            Vop_temp = Vop_copy[i]  # use the degraded voltage
                            temp_EC.append(np.nan)  # make EC nan
                            temp_degraded_voltage.append(Vop_copy[i])
                        else:  # threshold not reached
                            Vop_temp = Vop_copy[i]  # use the degraded voltage
                            temp_EC.append(
                                price * updatePReq(i, Jop[i], Vop_copy[i]))  # calculate Energy Charge with degraded voltage
                            temp_degraded_voltage.append(Vop_copy[i]) #for printing                  
                        if Jop[
                            current_index] not in threshold_list_J:  # if the threshold occurred, if I already added the Jop DON'T ADD IT AGAIN
                            threshold_list_J.append(Jop[current_index])
                            threshold_list_hour.append(hour)
                        Vop_plotting.append(Vop[i])
                        Vop_fallback.append(Vop_temp)
                        Jop_plotting.append(Jop[i])
                        Jop_fallback.append(Jop[current_index])
                    else:
                        temp_EC.append(
                            0)  
                        temp_degraded_voltage.append(Vop_copy[i])
                    Energy_Charge["EC_Jop" + str(round(Jop[i], 2))] = temp_EC  # collects Energy Charges in a dictionary
                    Degraded_Voltage["DV_Jop" + str(round(Jop[i], 2))] = temp_degraded_voltage
                else:  # anything other than thousands (e.g. 1 to 999 and 1001 to 1999, 2001 t0 2999)
                    if Jop[i] != 0:
                        if Vop_copy[i] >= V_threshold:
                            temp_EC.append(np.nan)
                            temp_degraded_voltage.append(Vop_copy[i])
                        else:
                            temp_EC.append(price * updatePReq(i, Jop[i], Vop_copy[i]))
                            temp_degraded_voltage.append(Vop_copy[i])         
                    else:
                        temp_EC.append(0)  
                        temp_degraded_voltage.append(Vop_copy[i])
                    Energy_Charge["EC_Jop" + str(round(Jop[i], 2))] = temp_EC
                    Degraded_Voltage["DV_Jop" + str(round(Jop[i], 2))] = temp_degraded_voltage

        df_LCOH = pd.DataFrame(data=Energy_Charge)  
        df_LCOH_total_list = []  
        start = 0
        stop = Life_1year
        for i in range(10):
            df_LCOH_temp = df_LCOH.iloc[start:stop]
            df_LCOH_total_list.append(df_LCOH_temp)  
            start = stop
            stop = (i + 2) * Life_1year  
        for df in df_LCOH_total_list:
            df.index = range(Life_1year)
    
        
        #CAPEX based on second current; USE Jop[2] FOR SECOND CURRENT
        if Jop[2] > 1.7:
            CAPEX_Electronics_Total =  CAPEX_Electronics*A*Jop[2]*Vop[2]/1000
            Installation_Factor = 1.19
            Total_CAPEX = (CAPEX_Rest_Total + CAPEX_Electronics_Total)*Installation_Factor
            M_CAPEX = (Total_CAPEX/Life_hours)
        elif Jop[2] > 0 and Jop[2] <= 1.7:
            CAPEX_Electronics_Total =  CAPEX_Electronics*A*Jop_rated*Vop_rated/1000
            Installation_Factor = 1.19
            Total_CAPEX = (CAPEX_Rest_Total + CAPEX_Electronics_Total)*Installation_Factor
            M_CAPEX = (Total_CAPEX/Life_hours) 
        else:
            M_CAPEX = 0  
        
        M_OM = 0.05*10*M_CAPEX 

        # Rate of fuel production in kg every hour of year 1
        M_kgh2 = []
        for i in range(len(Jop)):
            if Jop[i] != 0:
                M_kgh2.append((Jop[i] * A) / (n_mol * F) * (0.002 * 3600))  # hourly kg H2
            else:
                M_kgh2.append(0)  

        for df_LCOH in df_LCOH_total_list: 
            for i in range(len(Jop)):
                df_LCOH["kg_Jop" + str(round(Jop[i], 2))] = [np.nan if np.isnan(df_LCOH["EC_Jop" + str(round(Jop[i], 2))][j]) \
                                                                else M_kgh2[i] for j in range(
                    Life_1year)]  # 10.27.20 - list, if EC for that hour and current is NaN, make Nan, else use the calc
                df_LCOH["CAP_Jop" + str(round(Jop[i], 2))] = [np.nan if np.isnan(df_LCOH["EC_Jop" + str(round(Jop[i], 2))][j]) \
                                                                  else M_CAPEX for j in range(Life_1year)]
                df_LCOH["OM_Jop" + str(round(Jop[i], 2))] = [np.nan if np.isnan(df_LCOH["EC_Jop" + str(round(Jop[i], 2))][j]) \
                                                                else M_OM for j in range(Life_1year)]
        col1 = []
        for df_LCOH in df_LCOH_total_list:
            for i in range(len(Jop)):  #2.3.21 - 10.9.20 - length now 91
                CAP_name = 'CAP_Jop' + str(round(Jop[i], 2))
                OM_name = 'OM_Jop' + str(round(Jop[i], 2))
                EC_name = 'EC_Jop' + str(round(Jop[i], 2))
                kg_name = 'kg_Jop' + str(round(Jop[i], 2))

                col1.append(CAP_name)
                col1.append(OM_name)
                col1.append(EC_name)
                col1.append(kg_name)

            df_LCOH = df_LCOH[col1]

        # COMPARE ALL COLUMNS AND PICK UP THE MIN mlCOH
        for df_LCOH in df_LCOH_total_list: 
            for i in range(len(Jop)):
                if i != 0:
                    df_LCOH["mLCOH_Jop" + str(round(Jop[i], 2))] = (df_LCOH["CAP_Jop" + str(round(Jop[i], 2))] + df_LCOH[\
                        "OM_Jop" + str(round(Jop[i], 2))] + df_LCOH["EC_Jop" + str(round(Jop[i], 2))]) / df_LCOH[\
                                                                      "kg_Jop" + str(round(Jop[i], 2))]
                else:
                    df_LCOH["mLCOH_Jop" + str(round(Jop[i], 2))] = 10000  # zero never selected

            mLCOH_title_list = []

            for i in range(len(Jop)):  
                mLCOH_title_list.append("mLCOH_Jop" + str(round(Jop[i], 2)))

            df_LCOH["mLCOH_min"] = df_LCOH[mLCOH_title_list].min(axis=1)
            
            df_LCOH[mLCOH_title_list].loc[
                0].min()
            df_LCOH

        # sum the EC associated with the mLCOH for the annnual LCOH
        OptimalEC_List_DCFA = []  #- for all 10 years
        for df_LCOH in df_LCOH_total_list:
            OptimalEC_Year1 = 0  #makes for EACH year new columns
            OptimalJop_Year1_List = []
            OptimalEC_Year1_List = []

            for i in range(len(df_LCOH["mLCOH_Jop0.1"])):  
                mLCOH_list = []
                for k in range(len(Jop)): 
                    mLCOH_list.append(df_LCOH["mLCOH_Jop" + str(round(Jop[k], 2))][i])
                temp_mLCOH_min = min(mLCOH_list)

                for j in range(len(mLCOH_list)):
                    if temp_mLCOH_min == mLCOH_list[j]:  
                        index = j  
                if i == 0:  
                    pass
                OptimalEC_Year1 += df_LCOH["EC_Jop" + str(round(Jop[index], 2))][i]  
                OptimalEC_Year1_List.append(df_LCOH["EC_Jop" + str(round(Jop[index], 2))][i])  
                OptimalJop_Year1_List.append(Jop[index])

            OptimalEC_List_DCFA.append(OptimalEC_Year1) 

            df_LCOH["m_ECmin"] = OptimalEC_Year1_List  
            df_LCOH["OptimalJop"] = OptimalJop_Year1_List

        # for the minimum find the corresponding kg, put in a new column and sum
        OptimalKG_List_DCFA = []  # 10.13.20
        for df_LCOH in df_LCOH_total_list:
            OptimalKG_Year1 = 0
            # cycle through each row and find the minimum LCOH
            OptimalKG_Year1_List = []
            for i in range(len(df_LCOH["mLCOH_Jop0.1"])): #2.3.21
                mLCOH_list = []
                for k in range(len(Jop)):  #2.3.21 k collects 91 values into list (instead of 3)
                    mLCOH_list.append(df_LCOH["mLCOH_Jop" + str(round(Jop[k], 2))][i])
                temp_mLCOH_min = min(mLCOH_list)

                # then for the minimum LCOH find the location inside the list
                for j in range(len(mLCOH_list)):
                    if temp_mLCOH_min == mLCOH_list[j]:  # take the index from the min LCOH
                        index = j  # save the position for which it is associated so it can be used next line
                if i == 0:  # FOR TESTING
                    pass
                OptimalKG_Year1 += df_LCOH["kg_Jop" + str(round(Jop[index], 2))][i]  # sums the corresponding min kg
                OptimalKG_Year1_List.append(df_LCOH["kg_Jop" + str(round(Jop[index], 2))][i])  # adds all to list to check

            OptimalKG_List_DCFA.append(OptimalKG_Year1)  

            df_LCOH["m_kgmin"] = OptimalKG_Year1_List

        for df_LCOH in df_LCOH_total_list:  # 10.13.20
            # CAPEX YEAR 1
            CAPEX_Year1 = df_LCOH["CAP_Jop0.1"].sum()

            # OM YEAR 1
            OM_Year1 = df_LCOH["OM_Jop"+str(round(Jop[2],2))].sum()

            # LCOH MIN for year 1
            LCOHmin_Year1 = (CAPEX_Year1 + OM_Year1 + OptimalEC_Year1) / OptimalKG_Year1

        df_DCFA = pd.DataFrame(data={})
        df_DCFA["Year"] = list(range(0, 11))

        df_DCFA["CAPEX"] = [Total_CAPEX if i == 0 else 0 for i in range(11)]

        df_DCFA["PWF"] = [1 / (1 + DR) ** df_DCFA["Year"][i] for i in range(11)]

        # OM
        df_DCFA["PV_OM"] = [0 if i == 0 else (OM_Year1 * df_DCFA["PWF"][i]) for i in range(11)]

        # EC
        OptimalEC_List_DCFA.insert(0, 0)
        df_DCFA["PV_EC"] = [0 if i == 0 else (OptimalEC_List_DCFA[i] * df_DCFA["PWF"][i]) for i in range(11)]

        # kg
        OptimalKG_List_DCFA.insert(0, 0)
        df_DCFA["PV_KG"] = [0 if i == 0 else (OptimalKG_List_DCFA[i] * df_DCFA["PWF"][i]) for i in range(11)]

        # CALCULATE LCOH

        PV_Costs = (df_DCFA["CAPEX"].sum() + df_DCFA["PV_OM"].sum() + df_DCFA[
            "PV_EC"].sum())  

        PV_KG = df_DCFA["PV_KG"].sum()

        Lifetime_LCOH_dynamic_Jop0baseline = PV_Costs / PV_KG

        # CONTRIBUTION ANALYSIS
        CAPEX_Cont = df_DCFA["CAPEX"].sum() / PV_Costs * 100

        OM_Cont = df_DCFA["PV_OM"].sum() / PV_Costs * 100

        EC_Cont = df_DCFA["PV_EC"].sum() / PV_Costs * 100


        Lifetime_LCOH_dynamic_Jop0baseline, CAPEX_Cont, OM_Cont, EC_Cont
        
        df_DCFA["PV_KG"].sum()

        #Average optimal Jop for 10 years

        OptimalJop_mean = []
        for year in range(10):
            OptimalJop_mean.append(df_LCOH_total_list[year]["OptimalJop"].mean())
        Average_Jop_10yr = sum(OptimalJop_mean) / len(OptimalJop_mean) 
        Year1_Jop = OptimalJop_mean[0]
        Year10_Jop = OptimalJop_mean[9]


        print("INST_CAPEX:", df_DCFA["CAPEX"].sum())
        print("PV_OM:", df_DCFA["PV_OM"].sum()) 
        print("PV_KG:" , df_DCFA["PV_KG"].sum())
        print("PV_EC:", df_DCFA["PV_EC"].sum())
        print("Average Jop:", Average_Jop_10yr)
        print("Jop Frequency:", df_LCOH["OptimalJop"].value_counts())
        print("LCOH:", Lifetime_LCOH_dynamic_Jop0baseline)
        print(mean,second_current,Average_Jop_10yr,Lifetime_LCOH_dynamic_Jop0baseline, df_DCFA["CAPEX"].sum(), df_DCFA["PV_EC"].sum(), df_DCFA["PV_OM"].sum(), PV_Costs, PV_KG, file=f_file,sep=",") #1.9.21 - write to csv as 3 columns
f_file.close()
df_csv = pd.read_csv("figure2b.csv")

Mean Price: 0.0
second current: 2.3000000000000007
INST_CAPEX: 9972844.753113482
PV_OM: 3063940.691014031
PV_KG: 15539497.23208283
PV_EC: -6422073.08597262
Average Jop: 2.292691780821542
Jop Frequency: 2.3    8727
0.1      33
Name: OptimalJop, dtype: int64
LCOH: 0.4256709376992046
second current: 2.400000000000001
INST_CAPEX: 10147203.04726868
PV_OM: 3117508.5029576696
PV_KG: 16212176.323809866
PV_EC: -6969353.9712907635
Average Jop: 2.3919657534247754
Jop Frequency: 2.4    8727
0.1      33
Name: OptimalJop, dtype: int64
LCOH: 0.3883104558695162
second current: 2.500000000000001
INST_CAPEX: 10324030.627700891
PV_OM: 3171834.9496629257
PV_KG: 16884831.81108698
PV_EC: -7521410.886325957
Average Jop: 2.491260273972604
Jop Frequency: 2.5    8726
0.1      34
Name: OptimalJop, dtype: int64
LCOH: 0.3538356057011413
second current: 2.600000000000001
INST_CAPEX: 10503327.494410114
PV_OM: 3226920.0311297984
PV_KG: 17558488.72237038
PV_EC: -8010549.503579437
Average Jop: 2.5906963470317463
Jop Fr